In [ ]:
%run main.py
%load_ext autoreload
%autoreload 2

!mkdir -p {RAW_DIR}
s3 = S3()

In [ ]:
!wget https://storage.yandexcloud.net/natasha-corus/taiga/Fontanka.tar.gz -P {RAW_DIR}
!wget https://storage.yandexcloud.net/natasha-corus/ods/gazeta_v1.csv.zip -P {RAW_DIR}
!wget https://storage.yandexcloud.net/natasha-corus/ods/interfax_v1.csv.zip -P {RAW_DIR}
!wget https://storage.yandexcloud.net/natasha-corus/lenta-ru-news.csv.gz -P {RAW_DIR}
!wget https://storage.yandexcloud.net/natasha-corus/buriy/news-articles-2014.tar.bz2 -P {RAW_DIR}
!wget https://storage.yandexcloud.net/natasha-corus/buriy/news-articles-2015-part1.tar.bz2 -P {RAW_DIR}
!wget https://storage.yandexcloud.net/natasha-corus/buriy/news-articles-2015-part2.tar.bz2 -P {RAW_DIR}

In [ ]:
LOADS = {
    'gazeta_v1.csv.zip': load_ods_gazeta,
    'interfax_v1.csv.zip': load_ods_interfax,
    'Fontanka.tar.gz': load_taiga_fontanka,
    'lenta-ru-news.csv.gz': load_lenta,
    'news-articles-2015-part1.tar.bz2': load_buriy_news,
    'news-articles-2015-part2.tar.bz2': load_buriy_news,
    'news-articles-2014.tar.bz2': load_buriy_news,
}


lines = []  # Requires 15Gb RAM
for name in listdir(RAW_DIR):
    path = join_path(RAW_DIR, name)
    records = LOADS[name](path)
    for record in log_progress(records, desc=name):
        line = re.sub('\s+', ' ', record.text)  # news article -> single line
        lines.append(line)

In [ ]:
seed(1)
shuffle(lines)

In [ ]:
cap = 1000
dump_lines(lines[:cap], TEST)
dump_lines(log_progress(lines[cap:]), TRAIN)

In [ ]:
s3.upload(TEST, S3_TEST)
s3.upload(TRAIN, S3_TRAIN)